In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, optimizers

### Download the Data and Perform a Control Check

This code calculates the brightness (luminance) of each color and compares the average brightness by label
(0 = dark, 1 = light).

It checks whether colors labeled as light actually have higher brightness values than those labeled as dark

In [3]:
df = pd.read_csv("data/data_a2_mc1_vta_hs25.csv", sep=";")
lum = 0.2126*df.RED + 0.7152*df.GREEN + 0.0722*df.BLUE
print(lum.groupby(df.LIGHT_OR_DARK_FONT_IND).mean())

LIGHT_OR_DARK_FONT_IND
0     81.349917
1    187.567306
dtype: float64


In [5]:
X = df[["RED", "GREEN", "BLUE"]].to_numpy(dtype=float) / 255.0  # scale RGB to [0,1]
y = df["LIGHT_OR_DARK_FONT_IND"].astype(int).to_numpy().reshape(-1, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Define a simple sequential model
model = keras.Sequential([
    layers.Input(shape=(3,)),          # 3 inputs: R, G, B
    layers.Dense(4, activation='relu'),  # hidden layer with 4 neurons
    layers.Dense(1, activation='sigmoid')  # output: 0 (dark) or 1 (light)
])

In [25]:
# Compile the model
model.compile(
    optimizer=optimizers.SGD(learning_rate=0.1, momentum=0.0),  # <- SGD
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [26]:
n = len(X_train)
batch_size = 1
steps_per_epoch = int(np.ceil(n / batch_size))
target_steps = 100_000
epochs_needed = int(np.ceil(target_steps / steps_per_epoch))

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs_needed,
    batch_size=batch_size,
    verbose=1
)

Epoch 1/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - accuracy: 0.9275 - loss: 0.1820 - val_accuracy: 0.9257 - val_loss: 0.1230
Epoch 2/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 768us/step - accuracy: 0.9498 - loss: 0.1009 - val_accuracy: 0.9665 - val_loss: 0.0755
Epoch 3/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.9572 - loss: 0.0918 - val_accuracy: 0.9665 - val_loss: 0.0891
Epoch 4/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 756us/step - accuracy: 0.9563 - loss: 0.0924 - val_accuracy: 0.9294 - val_loss: 0.1195
Epoch 5/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.9610 - loss: 0.0809 - val_accuracy: 0.9628 - val_loss: 0.0645
Epoch 6/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.9647 - loss: 0.0665 - val_accuracy: 0.9926 - val_loss: 0.0377
Epoch 7/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step - accuracy: 0.9693 - loss: 0.0643 - val_accuracy: 0.9851 - val_loss: 0.0477
Epoch 8/93
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.9628 -

In [31]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.3f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9703 - loss: 0.0448 
Test accuracy: 0.970


In [28]:
def predict_color_class(model, rgb):
    """
    Predicts whether a color should use a light or dark font.
    
    Parameters:
        model: trained Keras model
        rgb: list or array of [R, G, B] values (0–255)
    
    Returns:
        "hell" if bright background (light font recommended)
        "dunkel" if dark background (dark font recommended)
    """
    sample = np.array(rgb).reshape(1, 3) / 255.0
    pred = model.predict(sample, verbose=0)
    return "hell" if pred > 0.5 else "dunkel"

In [ ]:
print(predict_color_class(model, [255, 255, 255]))  # hell
print(predict_color_class(model, [0, 0, 0]))        # dunkel
print(predict_color_class(model, [120, 120, 120]))  


hell
dunkel
dunkel
